In [ ]:
# Linear Regression with Numpy
We compute the regression line linking counts of letters in a text to counts of letter _a_. 

Book version 2024

Author: Pierre Nugues

## Modules
We import the modules we need to run the program

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

## The Dataset

We load the data and we possibly normalize it

The English dataset

Number of characters

In [ ]:
X = [[1, 35680.0],
     [1, 42514.0],
     [1, 15162.0],
     [1, 35298.0],
     [1, 29800.0],
     [1, 40255.0],
     [1, 74532.0],
     [1, 37464.0],
     [1, 31030.0],
     [1, 24843.0],
     [1, 36172.0],
     [1, 39552.0],
     [1, 72545.0],
     [1, 75352.0],
     [1, 18031.0]]

Number of a's

In [ ]:
y = [2217.0,
     2761.0,
     990.0,
     2274.0,
     1865.0,
     2606.0,
     4805.0,
     2396.0,
     1993.0,
     1627.0,
     2375.0,
     2560.0,
     4597.0,
     4871.0,
     1119.0]

### Function to normalize the data

We divide by the maximal value of the column

In [ ]:
def normalize(Xy):
    maxima = np.amax(Xy, axis=0)
    Xy = 1/maxima * Xy
    return (Xy, maxima)

We normalize

In [ ]:
normalized = True
debug = False
# Predictors
X = np.array(X)
# Response
y = np.array([y]).T

alpha = 1.0e-10
if normalized:
    X, maxima_X = normalize(X)
    y, maxima_y = normalize(y)
    maxima = np.concatenate((maxima_X, maxima_y))
    alpha = 1.0
    print("-Normalized-")

In [ ]:
X, y

## Function to compute the sum of squared errors

In [ ]:
def sse(X, y, w):
    error = y - X @ w
    return error.T @ error

## Function to predict values

In [ ]:
def predict(X, w):
    return X @ w

## Batch descent
Function to apply a batch descent

In [ ]:
def fit_batch(X, y, alpha, w,
              epochs=500,
              epsilon=1.0e-5):
    """
    Batch gradient descent
    :param X:
    :param y:
    :param alpha:
    :param w:
    :param epochs:
    :param epsilon:
    :return:
    """
    global logs
    logs = []
    alpha /= len(X)
    for epoch in range(epochs):
        error = predict(X, w) - y
        gradient = X.T @ error
        w = w - alpha * gradient
        logs += (w, alpha, sse(X, y, w))
        if np.linalg.norm(gradient) < epsilon:
            break
    print("Epoch", epoch)
    return w

## We apply a batch descent

In [ ]:
print("===Batch descent===")
w = np.zeros((X.shape[1], 1))
w = fit_batch(X, y, alpha, w)
print("Weights", w)
print("SSE", sse(X, y, w))
if normalized:
    maxima = maxima.reshape(-1, 1)
    w = maxima[-1, 0] * (w / maxima[:-1, 0:1])
    print("Restored weights", w)
if debug:
    print("Logs", logs)

## We restore the coordinates

In [ ]:
x_fig = [X[i][1] * maxima_X[1] for i in range(len(X))]
y_fig = [yi * maxima_y for yi in y]

## We plot the coordinates and the line

In [ ]:
plt.scatter(x_fig, y_fig)
plt.plot([min(x_fig), max(x_fig)],
         [[1, min(x_fig)] @ w, [1, max(x_fig)] @ w])
plt.show()

## Showing the iterations

### The errors

In [ ]:
plt.scatter(range(len(logs[2::3])), logs[2::3], c='b', marker='x')
plt.title("Batch gradient descent: Sum of squared errors")
plt.show()

In [ ]:
plt.plot(list(map(lambda pair: pair[0], logs[0::3])),
         list(map(lambda pair: pair[1], logs[0::3])), marker='o')
for i in range(len(logs[0::3])):
    plt.annotate(i, xy=logs[0::3][i])
plt.title("Batch gradient descent: Weights")
plt.show()

## Stochastic descent
Function to apply a stochastic descent

In [ ]:
def fit_stoch(X, y, alpha, w,
              epochs=500,
              epsilon=1.0e-5):
    """
    Stochastic gradient descent
    :param X:
    :param y:
    :param alpha:
    :param w:
    :param epochs:
    :param epsilon:
    :return:
    """
    global logs, logs_stoch
    logs = []
    logs_stoch = []
    random.seed(0)
    idx = list(range(len(X)))
    for epoch in range(epochs):
        random.shuffle(idx)
        for i in idx:
            # error is a scalar
            error = (predict(X[i], w) - y[i])[0]
            gradient = error * X[i:i + 1].T
            w = w - alpha * gradient
            logs_stoch += (w, alpha, sse(X, y, w))
        if np.linalg.norm(gradient) < epsilon:
            break
        logs += (w, alpha, sse(X, y, w))
    print("Epoch", epoch)
    return w

## We apply a stochastic descent

In [ ]:
print("===Stochastic descent===")
w = np.zeros((X.shape[1], 1))
w = fit_stoch(X, y, alpha, w)
print("Weights", w)
print("SSE", sse(X, y, w))
if normalized:
    maxima = maxima.reshape(-1, 1)
    w = maxima[-1, 0] * (w / maxima[:-1, 0:1])
    print("Restored weights", w)
if debug:
    print("Logs", logs)
    print("Logs stoch.", logs_stoch)

## We plot the line

In [ ]:
plt.scatter(x_fig, y_fig)
plt.plot([min(x_fig), max(x_fig)],
         [[1, min(x_fig)] @ w, [1, max(x_fig)] @ w])
plt.show()

## Showing the iterations

### The errors by epoch

In [ ]:
plt.scatter(range(len(logs[2::3])), logs[2::3], c='b', marker='x')
plt.title("Stochastic gradient descent: Sum of squared errors")
plt.show()

### The weight updates by epoch

In [ ]:
plt.plot(list(map(lambda pair: pair[0], logs[0::3])), list(
    map(lambda pair: pair[1], logs[0::3])), marker='o')
for i in range(len(logs[0::3])):
    plt.annotate(i, xy=logs[0::3][i])
plt.title("Stochastic gradient descent: Weights")
plt.show()

### The weight updates

In [ ]:
plt.scatter(range(len(logs_stoch[2::3])), logs_stoch[2::3], c='b', marker='x')
plt.title("Stochastic gradient descent: Sum of squared errors (individual updates)")
plt.show()

### The weight updates by individual update

In [ ]:
plt.plot(list(map(lambda pair: pair[0], logs_stoch[0::3])),
         list(map(lambda pair: pair[1], logs_stoch[0::3])),
         marker='o')
plt.title("Stochastic gradient descent: Weights (individual updates)")
plt.show()